In [114]:
import xmltodict, json, feedparser
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import time, re
import pandas as pd
from collections import defaultdict
import json

In [163]:
df = pd.read_json('calendarDB.json', orient='index')
df

,파크시스템스,파크시스템,파크시스,삼성전자,현대차,기아
2022-08-24,"[기업설명회(IR)개최, 반기실적 및 경영현황 업데이트]","[기업설명회(IR)개, 반기실적 및 경영현황 업데이트]","[기업설명회(IR), 반기실적 및 경영현황 업데이트]",NaN,NaN,NaN
2022-08-25,NaN,NaN,NaN,"[기업설명회(IR)개최, 반기실적 및 경영현황 업데이트]","[기업설명회(IR)개, 반기실적 및 경영현황 업데이트]","[기업설명회(IR), 반기실적 및 경영현황 업데이트]"


In [ ]:
def makeCalendar():
  
  while True:
    
    class BreakIt(Exception): pass
    df = pd.read_csv('blogRss_list.csv', index_col=0)
    
    for address in df['address']:
      rss_info = feedparser.parse(address) # 문법적으로 분석해서 사전으로 만듦
      disclosure_list = rss_info['entries']

      try:
        for disclosure in disclosure_list:
          link = disclosure['link']

          with open('./blogRss_dr.csv', 'r', encoding='utf-8-sig') as f: 
            reader = csv.reader(f, delimiter=',', quoting=csv.QUOTE_NONE) 
            for line in reader:
              line = ''.join(line) # .strip() -> ['https://blog.naver.com/jeunkim/222819001533'] 에서 [''] 삭제
              if line == link:
                print('중복입니다!')
                raise BreakIt
              else:
                continue  #Loop를 계속 돌려서 중복이 모든 line에 없으면 아래로 이동
                
            bot.send_message(chat_id=CHAT_ID, text=link)  ##새로운 블로그 추가할 때 주석처리
            bot2.sendMessage(chat_id='@DART_bok', text=link) ##테스트할 때 항상 주석처리
            print('항목 추가 완료')




          with open('./blogRss_dr.csv', 'a', encoding='utf-8-sig') as f: 
            f.write(link + '\n')
            f.close()


      except BreakIt:
        pass

    print(datetime.now())
    print('----------------------------------------------------------')
    time.sleep(600)
        

for i in range(1, 100):
  try:
    blogRss()
  except Exception as e:
    print(e)
    print('restarting')
    continue
  else:
    break

In [231]:
with open('calendarDB.json', encoding='utf-8') as db:
    res = json.load(db)


seen = ((date, res[date]['기업명'], res[date]['내용']) for date in res.keys())
# seen = (date for date in list(res.keys()))
set(seen)

{('2022-08-17', '3241', 'test3'), ('2022-08-24', '파크시스', '반기실적 및 경영현황 업데이트')}

In [242]:
companyName = 'CJ ENM'
title = '기업설명회(IR)'
# html = urlopen('https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220823900266')
# bsObj = BeautifulSoup(html, 'html.parser')

# driver = webdriver.Chrome(ChromeDriverManager().install())
option = Options()
option.add_argument('headless')
driver = webdriver.Chrome(options=option)
driver.get('https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220902900153')
# time.sleep(2)
html_source = driver.page_source


rcpNo = re.findall(r"openPdfDownload\('(.*?)', '", html_source)
dcmNo = re.findall(r"', '(.*?)'\);", html_source)

# url이 있어야 read_html을 통해 table 가져올 수 있다
frame_url = 'https://dart.fss.or.kr/report/viewer.do?rcpNo=' + rcpNo[0] + '&dcmNo=' + dcmNo[2][0:7] + '&eleId=' + '0' + '&offset=' + '0' + '&length=' + '0' + '&dtd=HTML'

table_ifrm = pd.read_html(frame_url, index_col=0)#, header=None)

# table_ifrm[0]
# date = defaultdict(dict)
# event = defaultdict(dict)

for index, row in table_ifrm[0].iterrows():
  if '목적' in index:
    text = row[2]
for index, row in table_ifrm[0].iterrows():
  if '일시' in index:
    if '20' in row[2]:
      dateNew = row[2]
    

with open('calendarDB.json', encoding='utf-8') as db:
  res = json.load(db)
print(res)
  
seen = ((date, res[date]['기업명'], res[date]['내용']) for date in res.keys())
seen = set(seen)

def add_entry(res, date, companyName, text):

    # check if in seen set
    if (date, companyName, text) in seen:
        return res

    # add to seen set
    seen.add(tuple([date, companyName, text]))

    # append to results list
    res[date] = {'기업명': companyName, '내용': text}
    # res.append({date:{'기업명': companyName, '내용': text}})

    return res

args = [dateNew, companyName, text]
# print(args)

res = add_entry(res, *args)  # add entry - SUCCESS
print(res)
res = add_entry(res, *args)  # try to add again - FAIL
# print(res)

# args2 = ['2022-08-17', '3241', 'test3']

res = add_entry(res, *args2)  # add another - SUCCESS
# print(res)


with open('calendarDB.json', 'w', encoding='utf-8') as db:
  json.dump(res, db, ensure_ascii=False)
db.close()

# date['행사일']
# for key,val in event.items():
#     if key in d:
#         d[key] = [d[key],val]

# event[1].append
# table_ifrm[0].loc['1.일시', '2']

seen

{'2022-08-24': {'기업명': '파크시스', '내용': '반기실적 및 경영현황 업데이트'}, '2022-08-17': {'기업명': '3241', '내용': 'test3'}, '2022-09-06': {'기업명': '파라다이스', '내용': '국내 기관투자자와의 커뮤니케이션'}}
{'2022-08-24': {'기업명': '파크시스', '내용': '반기실적 및 경영현황 업데이트'}, '2022-08-17': {'기업명': '3241', '내용': 'test3'}, '2022-09-06': {'기업명': 'CJ ENM', '내용': 'KB Korea Conference 2022'}}


{('2022-08-17', '3241', 'test3'),
 ('2022-08-24', '파크시스', '반기실적 및 경영현황 업데이트'),
 ('2022-09-06', 'CJ ENM', 'KB Korea Conference 2022'),
 ('2022-09-06', '파라다이스', '국내 기관투자자와의 커뮤니케이션')}

In [175]:
event

defaultdict(dict, {'파크시스': ['기업설명회(IR)', '반기실적 및 경영현황 업데이트']})

In [172]:
event2 = defaultdict(dict)
event2

for key, val in event2.items():
    if key in d:
        d[key] = [d[key],val]

defaultdict(dict, {})

In [171]:
date

defaultdict(dict,
            {'2022-08-24': defaultdict(dict,
                         {'파크시스템스': ['기업설명회(IR)개최', '반기실적 및 경영현황 업데이트'],
                          '파크시스템': ['기업설명회(IR)개', '반기실적 및 경영현황 업데이트'],
                          '파크시스': ['기업설명회(IR)', '반기실적 및 경영현황 업데이트']})})

In [166]:
table_ifrm[0]

,1,2,3,4,5
0,,,,,
1. 일시,1. 일시,행사일,행사일,시간(현지시간),시간(현지시간)
1. 일시,1. 일시,시작일,종료일,시작시간,종료시간
1. 일시,1. 일시,2022-08-24,2022-08-25,09:00,16:00
2. 장소,2. 장소,-,-,-,-
3. 대상자,3. 대상자,국내 기관투자자,국내 기관투자자,국내 기관투자자,국내 기관투자자
4. 실시목적,4. 실시목적,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트
5. 실시방법,5. 실시방법,컨퍼런스콜 형식의 NDR,컨퍼런스콜 형식의 NDR,컨퍼런스콜 형식의 NDR,컨퍼런스콜 형식의 NDR
6. 주요내용,6. 주요내용,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트,반기실적 및 경영현황 업데이트
7. 후원기관,7. 후원기관,BNK증권,BNK증권,BNK증권,BNK증권


In [165]:
date

defaultdict(dict,
            {'2022-08-24': defaultdict(dict,
                         {'파크시스템스': ['기업설명회(IR)개최', '반기실적 및 경영현황 업데이트'],
                          '파크시스템': ['기업설명회(IR)개', '반기실적 및 경영현황 업데이트'],
                          '파크시스': ['기업설명회(IR)', '반기실적 및 경영현황 업데이트']})})

In [26]:
rss_info = feedparser.parse('http://dart.fss.or.kr/api/todayRSS.xml')
disclosureList = rss_info['entries']
disclosureList[0]

{'title': '(코스닥)에스트래픽 - 주주명부폐쇄기간또는기준일설정',
 'title_detail': {'type': 'text/plain',
  'language': None,
  'base': 'https://dart.fss.or.kr/api/todayRSS.xml',
  'value': '(코스닥)에스트래픽 - 주주명부폐쇄기간또는기준일설정'},
 'links': [{'rel': 'alternate',
   'type': 'text/html',
   'href': 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220823900466'}],
 'link': 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220823900466',
 'tags': [{'term': '코스닥', 'scheme': None, 'label': None}],
 'published': 'Tue, 23 Aug 2022 09:16:00 GMT',
 'published_parsed': time.struct_time(tm_year=2022, tm_mon=8, tm_mday=23, tm_hour=9, tm_min=16, tm_sec=0, tm_wday=1, tm_yday=235, tm_isdst=0),
 'id': 'http://dart.fss.or.kr/api/link.jsp?rcpNo=20220823900466',
 'guidislink': False,
 'authors': [{'name': '에스트래픽'}],
 'author': '에스트래픽',
 'author_detail': {'name': '에스트래픽'},
 'updated': '2022-08-23T09:16:00Z',
 'updated_parsed': time.struct_time(tm_year=2022, tm_mon=8, tm_mday=23, tm_hour=9, tm_min=16, tm_sec=0, tm_wday=1, tm_yday=235, tm_isdst=0)}

In [11]:
with open('http://dart.fss.or.kr/api/todayRSS.xml') as dx:
  doc = xmltodict.parse(dx.read())

doc

OSError: [Errno 22] Invalid argument: 'https://dart.fss.or.kr/api/todayRSS.xml'

In [ ]:
print('hello again')

hello again
